# Media Matters Worldwide Bayesian Media Mix Model

This notebook contains a media mix model, created from methods similar to __[this paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46001.pdf)__. The model informs a budget optimizer, which optimizes the budget for spending in advertising media channels in the short-term.

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score 
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

from datetime import datetime as dt
from functools import reduce
from yellowbrick.regressor import residuals_plot
from transformations import transform_one_column, select_best_feature, feature_imp_random_forest

import time
import datetime
import ipywidgets as widgets
from ipywidgets import FileUpload

from IPython.display import display
from IPython.display import FileLink
import io
import re
from scipy.optimize import minimize, LinearConstraint

import openpyxl
import xlrd

import holidays

from helper import *
from bayesian import *

import panel as pn
pn.extension()

***

<a id="data-collection"></a> 
# Data Collection

### your inputs start

In [2]:
client = "Proactiv"
use_excel = True
granularity = True # TRUE CORRESPONDS TO DAILY
data_type = "SALES DATA"
sales_fileName = "Proactiv_OrderData_2022.03.23.xlsx" # TYPE THE FILE NAME OF THE SALES DATA
ad_fileName = "Proactiv_MediaData_2022.03.23.xlsx"# TYPE THE FILE NAME OF THE MEDIA DATA

### your inputs end

In [3]:
if use_excel:
    sales_df = pd.read_excel(sales_fileName)
else:
    sales_df = pd.read_csv(sales_fileName)

In [4]:
sales_df_cleaned = clean_sales(sales_df, data_type, granularity=granularity)

COLUMNS of SALES DATA: ['Campaign Geo' 'Day' 'Visits' 'Orders' 'Conv Rate (Adobe)'
 'Revenue (Adobe)']


COUNTRY | [column-name/NONE] Campaign Geo
DAY | [column-name] Day
RESPONSE VARIABLE | [column-name] Orders


This process will be repeated for the advertising media mix data.

In [5]:
if use_excel:
    ad_df = pd.read_excel(ad_fileName)
else:
    ad_df = pd.read_csv(ad_fileName)

In [6]:
data_dict = clean_and_merge(ad_df, sales_df_cleaned)
for key in data_dict:
    data_dict[key].columns = [re.sub('[^0-9a-zA-Z_=.]+', '', col) for col in data_dict[key].columns]

['Day' 'Campaign Geo' 'Channel' 'Publisher' 'Media Objective' 'Audience'
 'Ad Type' 'Media Cost' 'Impressions']


COUNTRY? Campaign Geo
MEDIA COST? Media Cost
IMPRESSIONS? Impressions


***

# Modeling

### your inputs start

In [7]:
country = "US"
target = 'Orders'

df = data_dict[country]

In [8]:
dfMediaCombined = pd.DataFrame()
fringe = set([])
cols = sorted(df.columns.values)
for col in get_media_vars(df):
    short = shorten_f_name(col)
    if short in fringe:
        dfMediaCombined[f"{short}_media_cost".lower()] += df[col]
    else:
        dfMediaCombined[f"{short}_media_cost".lower()] = df[col]
    fringe.add(col)

dfMediaCombined[target] = df[target]

In [9]:
dfMediaCombined.isna().sum()

amazon_media_cost          0
bingsearch_media_cost      0
ctv_media_cost             0
criteo_media_cost          0
dv360_media_cost           0
facebook_media_cost        0
googlesearch_media_cost    0
influential_media_cost     0
lineartv_media_cost        0
pinterest_media_cost       0
radio_media_cost           0
snapchat_media_cost        0
thetradedesk_media_cost    0
tinder_media_cost          0
twitch_media_cost          0
youtube_media_cost         0
Orders                     0
dtype: int64

### your inputs end

In [13]:
initial_model = BayesianMixModel(country=country, target=target)

X = dfMediaCombined.drop(columns=[target])[["youtube_media_cost", "googlesearch_media_cost", "facebook_media_cost"]]
y = dfMediaCombined[target]
xtrain, xval, ytrain, yval = train_test_split(X,y, test_size=0.1, shuffle=False)

In [14]:
initial_model.fit(xtrain, ytrain)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, base, car_facebook_media_cost, sat_facebook_media_cost, coef_facebook_media_cost, car_googlesearch_media_cost, sat_googlesearch_media_cost, coef_googlesearch_media_cost, car_youtube_media_cost, sat_youtube_media_cost, coef_youtube_media_cost]


Sampling 4 chains for 3_000 tune and 1_000 draw iterations (12_000 + 4_000 draws total) took 503 seconds.
There were 194 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9196111954546349, but should be close to 0.8. Try to increase the number of tuning steps.
There were 20 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8861238873282821, but should be close to 0.8. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


In [15]:
initial_model.attribution(target=target)

ValueError: cannot reshape array of size 1384000 into shape (4000,200)

In [ ]:
initial_model.lineplot()

In [ ]:
initial_model.scatterplot()

In [ ]:
trainPred = initial_model.predict(xtrain)
mape(ytrain, trainPred)

In [15]:
valPred = initial_model.predict(xval)
mape(yval, valPred)

I peed on deez nuts
I pissed in yo wallet, and then took a dump on yo grave


ValueError: Found input variables with inconsistent numbers of samples: [39, 346]

In [ ]:
plt.plot(np.arange(len(yval)), yval)

In [ ]:
with initial_model.mmm:
    pm.set_data({"data": xval}, model=initial_model.mmm)
    ppc_test = pm.sample_posterior_predictive(initial_model.trace, model=initial_model.mmm, samples=100)
    p_test_pred = ppc_test["sales"].mean(axis=0)
    

In [ ]:
len(p_test_pred)

# Forecasting

In [16]:
initial_model.trace

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data
	> constant_data

In [ ]:
X

In [46]:
X.columns

Index(['youtube_media_cost', 'googlesearch_media_cost', 'facebook_media_cost'], dtype='object')

To forecast for one-month (or any time length greater than 1 day), write an excel spreadsheet or .csv file with the above columns in that **EXACT** order. Set `use_excel` to `True` if the file is an excel file, else set it to `False`.

In [47]:
data_type = "ad data [forecasting]"
use_excel = True

In [48]:
forecast_as_bytes = input_file(data_type, use_excel=use_excel)

FileUpload(value={}, accept='.xlsx', description='Upload')

In [49]:
forecast_sales_df = parse_file(forecast_as_bytes, data_type, use_excel=use_excel)

AssertionError: PLEASE go back and run the above CELL

In [55]:
forecast_sales_df = forecast_sales_df.iloc[:, np.arange(len(get_media_vars(x)))]
forecast_sales_df.columns = get_media_vars(x)

In [56]:
forecast_output_df = month_predict(forecast_sales_df.copy(), data_dict[country].copy(), model, model_type)

REQUIRES EDIT ON BAYESIANMIXMODEL.PREDICT to change dataset before sampling; then the rest of the code should work automatically as it only depends on implementing sklearn's API.